In [ ]:
%reload_ext autoreload
%autoreload 2

from utils import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm

In [ ]:
df_test = read_data('data/test', 'test')

In [ ]:
plt.figure(figsize = (7,7))
sorted_counts = df_test['label_text'].value_counts()
plt.pie(sorted_counts, labels = sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.1f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 13}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'Total Tweets: {}'.format(df_test.shape[0]))
plt.title('Distribution of Tweets in the Dataset', fontsize = 16);

In [ ]:
X_test, y_test = df_test[['text']], df_test[['label']]

In [ ]:
from utils import *
from polishbert import *

# load embeddings and model

In [ ]:
get_embeddings, tokenizer = load_embeddings(model_dir = "models/roberta_base_transformers")

In [ ]:
from model import *

In [ ]:
def load_trained_model(net, model_path, device):
    net.load_state_dict(
            torch.load(model_path, map_location=device))
    print("[*] Loaded weights from {}".format(model_path))
        
    net = net.to(device)
    net.eval()
    return net

In [ ]:
embed_num = 30
embed_dim = 768
class_num = 3
kernel_num = 3
kernel_sizes = [2, 3, 4]
dropout = 0.5
kernel_num = 3
kernel_sizes = [2, 3, 4]
dropout = 0.3
static = True
model = KimCNN(
    embed_num=embed_num,
    embed_dim=embed_dim,
    class_num=class_num,
    kernel_num=kernel_num,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    static=static,
)

In [ ]:
model = load_trained_model(model, 'models/emb_roberta_kimcnn.pth', 'cpu')

# predictions

In [ ]:
#on cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
y_preds = []
model.eval() 
with torch.no_grad():
    for sentence in tqdm(X_test['text']):
        sentence = clean_tweet(sentence)
        test_indices = tokenize_and_pad_text(pd.DataFrame([sentence])[0], 30, tokenizer)
        embeddings = get_embeddings(test_indices)[0]
        y_pred = model(embeddings)
        y_pred = torch.argmax(y_pred, dim=1)
        y_preds.extend(y_pred.detach().numpy().tolist())
            #clean
y_preds_np = np.array(y_preds)

In [ ]:
performance_per_second = 74/1000 # 1000 examples, 1min14 seconds
performance_per_second

## Solution runs on CPU at 74 milliseconds per observation

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer

In [ ]:
y_test = np.array(y_test)

In [ ]:
f1_score(y_test, y_preds_np, average='macro')

In [ ]:
f1_score(y_test, y_preds_np, average='micro')

In [ ]:
recall_score(y_test, y_preds_np, average='macro')

In [ ]:
accuracy_score(y_test, y_preds_np)

![](docs/leaderboard.png)